# Operations problems with [PowerSimulations.jl](https://github.com/NREL-SIIP/PowerSimulations.jl)

**Originally Contributed by**: Clayton Barrows

## Introduction

PowerSimulations.jl supports the construction and solution of optimal power system
scheduling problems (Operations Problems). Operations problems form the fundamental
building blocks for [sequential simulations](https://nbviewer.jupyter.org/github/NREL-SIIP/SIIPExamples.jl/blob/master/notebook/3_PowerSimulations_examples/02_sequential_simulations.ipynb).
This example shows how to specify and customize a the mathematics that will be applied to the data with
an `OperationsProblemTemplate`, build and execute an `OperationsProblem`, and access the results.

## Dependencies

In [1]:
using SIIPExamples

### Modeling Packages

In [2]:
using PowerSystems
using PowerSimulations
const PSI = PowerSimulations
using PowerSystemCaseBuilder

### Data management packages

In [3]:
using Dates
using DataFrames

### Optimization packages

In [4]:
using Cbc #solver

### Data
This data depends upon the [RTS-GMLC](https://github.com/gridmod/rts-gmlc) dataset. Let's
use [PowerSystemCaseBuilder.jl](https://nbviewer.jupyter.org/github/NREL-SIIP/SIIPExamples.jl/blob/master/notebook/3_PowerSimulations_examples/10_PowerSystemCaseBuilder.ipynb) to download and build a `System`.

In [5]:
sys = build_system(PSITestSystems, "modified_RTS_GMLC_DA_sys")

┌ Info: Building new system modified_RTS_GMLC_DA_sys from raw data
└   sys_descriptor.raw_data = "/Users/cbarrows/Documents/repos/SIIPExamples.jl/notebook/3_PowerSimulations_examples/data/RTS-GMLC-master"
[ Info: Parsing csv data in branch.csv ...
[ Info: Successfully parsed branch.csv
[ Info: Parsing csv data in bus.csv ...
[ Info: Successfully parsed bus.csv
[ Info: Parsing csv data in dc_branch.csv ...
[ Info: Successfully parsed dc_branch.csv
[ Info: Parsing csv data in gen.csv ...
[ Info: Successfully parsed gen.csv
[ Info: Parsing csv data in reserves.csv ...
[ Info: Successfully parsed reserves.csv
[ Info: Parsing csv data in simulation_objects.csv ...
[ Info: Successfully parsed simulation_objects.csv
[ Info: Parsing csv data in storage.csv ...
[ Info: Successfully parsed storage.csv
[ Info: Parsing csv data in timeseries_pointers.csv ...
[ Info: Successfully parsed timeseries_pointers.csv
[ Info: Unit System changed to InfrastructureSystems.UnitSystemModule.UnitSystem.DEVICE_B

,ConcreteType,SuperTypes,Count
,String,String,Int64
1,Arc,Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,109
2,Area,AggregationTopology <: Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,3
3,Bus,Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,73
4,HydroDispatch,HydroGen <: Generator <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,1
5,Line,ACBranch <: Branch <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,105
6,LoadZone,AggregationTopology <: Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,21
7,PowerLoad,StaticLoad <: ElectricLoad <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,51
8,RenewableDispatch,RenewableGen <: Generator <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,29
9,RenewableFix,RenewableGen <: Generator <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,31


## Define a problem specification with an `OperationsProblemTemplate`
You can create an empty template with:

In [6]:
template_uc = OperationsProblemTemplate()


Operations Problem Specification
Transmission: PowerSimulations.CopperPlatePowerModel
Devices Models: 

Branches Models: 

Services Models:



Now, you can add a `DeviceModel` for each device type to create an assignment between PowerSystems device types
and the subtypes of `AbstractDeviceFormulation`. PowerSimulations has a variety of different
`AbstractDeviceFormulation` subtypes that can be applied to different PowerSystems device types,
each dispatching to different methods for populating optimization problem objectives, variables,
and constraints.

In [7]:
print_tree(PSI.AbstractDeviceFormulation)

AbstractDeviceFormulation
├─ AbstractBranchFormulation
│  ├─ AbstractDCLineFormulation
│  │  ├─ HVDCDispatch
│  │  ├─ HVDCLossless
│  │  └─ HVDCUnbounded
│  ├─ StaticBranch
│  ├─ StaticBranchBounds
│  └─ StaticBranchUnbounded
├─ AbstractHydroFormulation
│  ├─ AbstractHydroDispatchFormulation
│  │  ├─ AbstractHydroReservoirFormulation
│  │  │  ├─ HydroDispatchPumpedStorage
│  │  │  ├─ HydroDispatchPumpedStoragewReservation
│  │  │  ├─ HydroDispatchReservoirBudget
│  │  │  └─ HydroDispatchReservoirStorage
│  │  └─ HydroDispatchRunOfRiver
│  └─ AbstractHydroUnitCommitment
│     ├─ HydroCommitmentReservoirBudget
│     ├─ HydroCommitmentReservoirStorage
│     └─ HydroCommitmentRunOfRiver
├─ AbstractLoadFormulation
│  ├─ AbstractControllablePowerLoadFormulation
│  │  ├─ DispatchablePowerLoad
│  │  └─ InterruptiblePowerLoad
│  └─ StaticPowerLoad
├─ AbstractRegulationFormulation
│  ├─ DeviceLimitedRegulation
│  └─ ReserveLimitedRegulation
├─ AbstractRenewableFormulation
│  └─ AbstractRenewable

### Branch Formulations
Here is an example of relatively standard branch formulations. Other formulations allow
for selective enforcement of transmission limits and greater control on transformer settings.

In [8]:
set_device_model!(template_uc, Line, StaticBranch)
set_device_model!(template_uc, Transformer2W, StaticBranch)
set_device_model!(template_uc, TapTransformer, StaticBranch)

### Injection Device Formulations
Here we define template entries for all devices that inject or withdraw power on the
network. For each device type, we can define a distinct `AbstractDeviceFormulation`. In
this case, we're defining a basic unit commitment model for thermal generators,
curtailable renewable generators, and fixed dispatch (net-load reduction) formulations
for `HydroDispatch` and `RenewableFix` devices.

In [9]:
set_device_model!(template_uc, ThermalStandard, ThermalStandardUnitCommitment)
set_device_model!(template_uc, RenewableDispatch, RenewableFullDispatch)
set_device_model!(template_uc, PowerLoad, StaticPowerLoad)
set_device_model!(template_uc, HydroDispatch, FixedOutput)
set_device_model!(template_uc, HydroEnergyReservoir, HydroDispatchRunOfRiver)
set_device_model!(template_uc, RenewableFix, FixedOutput)

### Service Formulations
We have two `VariableReserve` types, parameterized by their direction. So, similar to
creating `DeviceModel`s, we can create `ServiceModel`s. The primary difference being
that `DeviceModel` objects define how constraints get created, while `ServiceModel` objects
define how constraints get modified.

In [10]:
set_service_model!(template_uc, VariableReserve{ReserveUp}, RangeReserve)
set_service_model!(template_uc, VariableReserve{ReserveDown}, RangeReserve)

### Network Formulations
Finally, we can define the transmission network specification that we'd like to model. For simplicity, we'll
choose a copper plate formulation. But there are dozens of specifications available through
an integration with [PowerModels.jl](https://github.com/lanl-ansi/powermodels.jl). *Note that
many formulations will require appropriate data and may be computationally intractable*

In [11]:
set_transmission_model!(template_uc, CopperPlatePowerModel)

## `OperationsProblem`
Now that we have a `System` and an `OperationsProblemTemplate`, we can put the two together
to create an `OperationsProblem` that we solve.

### Optimizer
It's most convenient to define an optimizer instance upfront and pass it into the
`OperationsProblem` constructor. For this example, we can use the free Cbc solver with a
relatively relaxed MIP gap (`ratioGap`) setting to improve speed.

In [12]:
solver = optimizer_with_attributes(Cbc.Optimizer, "logLevel" => 1, "ratioGap" => 0.5)

MathOptInterface.OptimizerWithAttributes(Cbc.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute, Any}[MathOptInterface.RawParameter("logLevel") => 1, MathOptInterface.RawParameter("ratioGap") => 0.5])

### Build an `OperationsProblem`
The construction of an `OperationsProblem` essentially applies an `OperationsProblemTemplate`
to `System` data to create a JuMP model.

In [13]:
op_problem = OperationsProblem(template_uc, sys; optimizer = solver, horizon = 24)

build!(op_problem, output_dir = mktempdir())

PowerSimulations.BuildStatusModule.BuildStatus.BUILT = 0

The principal component of the `OperationsProblem` is the JuMP model. For small problems,
you can inspect it by simply printing it to the screen:
```julia
op_problem.internal.optimization_container.JuMPmodel
```

For anything of reasonable size, that will be unmanageable. But you can print to a file:
```julia
f = open("testmodel.txt","w"); print(f,op_problem.internal.optimization_container.JuMPmodel); close(f)
```

In addition to the JuMP model, an `OperationsProblem` keeps track of a bunch of metadata
about the problem and some references to pretty names for constraints and variables.
All of these details are contained within the `optimization_container` field.

In [14]:
print_struct(typeof(op_problem.internal.optimization_container))

mutable struct PowerSimulations.OptimizationContainer
    JuMPmodel::JuMP.Model
    time_steps::UnitRange{Int64}
    resolution::Dates.TimePeriod
    settings::PowerSimulations.Settings
    settings_copy::PowerSimulations.Settings
    variables::Dict{Symbol, AbstractArray}
    aux_variables::Dict{PowerSimulations.AuxVarKey, AbstractArray}
    constraints::Dict{Symbol, AbstractArray}
    cost_function::JuMP.AbstractJuMPScalar
    expressions::Dict{Symbol, JuMP.Containers.DenseAxisArray}
    parameters::Dict{Symbol, PowerSimulations.ParameterContainer}
    initial_conditions::PowerSimulations.InitialConditions
    pm::Union{Nothing, PowerModels.AbstractPowerModel}
    base_power::Float64
    solve_timed_log::Dict{Symbol, Any}
end


### Solve an `OperationsProblem`

In [15]:
solve!(op_problem)

PowerSimulations.RunStatusModule.RunStatus.SUCCESSFUL = 0

## Results Inspection
PowerSimulations collects the `OperationsProblem` results into a struct:

In [16]:
print_struct(PSI.ProblemResults)

 struct PowerSimulations.ProblemResults
    base_power::Float64
    timestamps::StepRange{Dates.DateTime, Dates.Millisecond}
    system::Union{Nothing, PowerSystems.System}
    variable_values::Dict{Symbol, DataFrames.DataFrame}
    dual_values::Dict{Symbol, DataFrames.DataFrame}
    parameter_values::Dict{Symbol, DataFrames.DataFrame}
    optimizer_stats::PowerSimulations.OptimizerStats
    output_dir::String
end


In [17]:
res = ProblemResults(op_problem);

### Optimizer Stats
The optimizer summary is included

In [18]:
get_optimizer_stats(res)

PowerSimulations.OptimizerStats(0, 0, 1.5862451806025987e6, 1, 1, 0, 7.941517114639282, 15.829380801, 1.158917799e9, 0.78771383)

### Objective Function Value

In [19]:
get_objective_value(res)

1.5862451806025987e6

### Variable Values
The solution value data frames for variables can be accessed by:

In [20]:
variable_values = get_variables(res)

Dict{Symbol, DataFrames.DataFrame} with 10 entries:
  Symbol("Reg_Down__Variab… => 24×52 DataFrame…
  :P__ThermalStandard       => 24×55 DataFrame…
  :start__ThermalStandard   => 24×55 DataFrame…
  :P__RenewableDispatch     => 24×30 DataFrame…
  Symbol("Reg_Up__Variable… => 24×52 DataFrame…
  Symbol("Spin_Up_R1__Vari… => 24×17 DataFrame…
  Symbol("Spin_Up_R3__Vari… => 24×18 DataFrame…
  :stop__ThermalStandard    => 24×55 DataFrame…
  Symbol("Spin_Up_R2__Vari… => 24×19 DataFrame…
  :On__ThermalStandard      => 24×55 DataFrame…

## Plotting
Take a look at the examples in [the plotting folder.](../../notebook/3_PowerSimulations_examples/Plotting)

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*